
### Variables
1.  **Observations (Emissions):** The input string.
    * `C` (Charged)
    * `N` (Neutral)
    * `H` (Hydrophobic)
2.  **Hidden States:** The labels you want to predict.
    * `State 0`: Non-transmembrane (Outside the wall).
    * `State 1`: Transmembrane (Inside the wall).



In [27]:
with open('Sequence_case2.txt', 'r') as f:
  case = f.read()

protein_sequence = case.rsplit('\n')[1]
len_protein_sequence = len(protein_sequence)
print(protein_sequence)

status_transmembrane = case.rsplit('\n')[4]
len_status_transmembrane = len(status_transmembrane)
print(status_transmembrane)

# check if both are same length

print(len_protein_sequence)
print(len_status_transmembrane)

assert(len_protein_sequence == len_status_transmembrane)

NNCCNNHCNCNNCNNHNCHCCCHHNHHNNHHHNCHNNNHNHNNHCNNHHNCHHCCCCNNHHCNNCNNNNNNHNCNNHNNHNHNNNHNNNHHHCHHNHHCNNHNCHCCCNCNNCCCHNCNNCNNNHHHNHNCNHNHNNNHNCNNNNCNHHHCNNNNNNNNNHHNHHNNNNNHNCHNCCHNCCNHNCNHNHHHNNNHHNNHNNNHNCNHCCNCCNCHCNHHHHHHNHHHNNHNNNHHNHCCNCCCHHHCNNCCCCNCCHNNNCNNCHNHCHCNNNHHNNHNNNNHHHNNNNHNNCNNNNNNCCCHNNHNHNHCNNCNHNHHHNNNNHHNNHNNNNNHHNCNCCNNCCHCCNNNNHNNHCNNCCNNCNHCCHNNCHNHHNNHNHNNHHHHNCNNCCNNCHNNNNNNNNHHHHHHHHNCNNNCHCNHNNNNCNCNNHCCNCHCNNNC
00000000000000000000001111111111111111111110000000000000000000000000000000111111111111111111111000000000000000000000000001111111111111111111110000000000001111111111111111111110000000011111111111111111111100000000000011111111111111111111100000000000000000000000000000000011111111111111111111111111111111111111111100001111111111111111111110000000000000000000000000000000001111111111111111111110000011111111111111111111100000000000000000000000000
443
443


(1) Use occurrence counts to compute estimates for model parameters.

In [31]:
st_count: dict[str] = {}

st_prev: str = status_transmembrane[0]
for i in status_transmembrane[1:]:
  combination = st_prev + i

  if combination in st_count:
    st_count[combination] += 1
  else:
    st_count[combination] = 1

  st_prev = i

print(st_count)
print(sum(st_count.values()))

st_trans_prob: dict[str, float] = {}
total: float = 0.0

for key, value in st_count.items():
  st_trans_prob[key] = value / len_status_transmembrane
  total += value / len_status_transmembrane

print(total)
print(st_trans_prob)


{'00': 201, '01': 10, '11': 221, '10': 10}
442
0.9977426636568849
{'00': 0.45372460496614, '01': 0.022573363431151242, '11': 0.49887133182844245, '10': 0.022573363431151242}


In [30]:
ps_count: dict[int] = {}

ps_prev: str = protein_sequence[0]
for i in protein_sequence[1:]:
    combination = ps_prev + i

    if combination in ps_count:
        ps_count[combination] += 1

    else:
        ps_count[combination] = 1
    
    ps_prev = i
    
print(ps_count)
print(sum(ps_count.values()))

ps_trans_prob: dict[float] = {}
total: float = 0.0

for (key, value) in ps_count.items():
    ps_trans_prob[key] = value / len_protein_sequence
    total += value / len_protein_sequence

print(total)
print(ps_trans_prob)

{'NN': 108, 'NC': 46, 'CC': 30, 'CN': 46, 'NH': 61, 'HC': 22, 'HN': 60, 'CH': 21, 'HH': 48}
442
0.9977426636568849
{'NN': 0.24379232505643342, 'NC': 0.1038374717832957, 'CC': 0.06772009029345373, 'CN': 0.1038374717832957, 'NH': 0.13769751693002258, 'HC': 0.04966139954853273, 'HN': 0.13544018058690746, 'CH': 0.04740406320541761, 'HH': 0.10835214446952596}


In [69]:
hmm_emission_prob = {}

for i, v in enumerate(protein_sequence):
    combination = v + '_' + status_transmembrane[i]
    if combination not in hmm_emission_prob:
        hmm_emission_prob[combination] = 1
        continue
    
    hmm_emission_prob[combination] += 1  



print(hmm_emission_prob)

assert(sum(hmm_emission_prob.values()) == len_protein_sequence == len_status_transmembrane)


{'N_0': 88, 'C_0': 84, 'H_0': 40, 'H_1': 90, 'N_1': 127, 'C_1': 14}


(2) Implement the Viterbi algorithm and use it to determine the prediction
accuracy of your model (in terms of numbers of correctly and incorrectly
annotated amino acids).

(3) Given the constructed HMM calculate the conditional probabilities (display
the result in matrix-form: P (C I C), P(C I N), P(C I H), … P(H I H).


(4) Change the emission probabilities to radically different values. Is it easier or
more difficult to annotate the hidden states with certainty? How could you
change the emission probabilities in order to reduce the HMM to a simple
Markov chain model? Define the resulting model.